## Quick start demo

### add TileDB.Cloud package, other dependency packages (Newtonsoft.Json, RestSharp, Polly, TileDB.CSharp) will also be added

In [ ]:
#r "nuget:TileDB.Cloud"

### Login

In [ ]:
string host = "https://api.tiledb.com"; //default host
string api_key = System.Environment.GetEnvironmentVariable("REST_TOKEN_KEY"); //default token
TileDB.Cloud.Client.Login(token:api_key,host:host);
//TileDB.Cloud.Client.Login(); // use default values

### Get User information

In [ ]:
var user_details = TileDB.Cloud.RestUtil.GetUser();
System.Console.WriteLine("{0}", user_details.ToJson());

### Array uri infos

In [ ]:
string original_file = "s3://tiledb-inc-demo-data/files/original/VLDB17_TileDB.pdf";

//TileDB File details
string tiledb_file_name = "VLDB17_TileDB_Example";
string tiledb_file_s3_uri = user_details.DefaultS3Path + "/" + tiledb_file_name;
System.Console.WriteLine("tiledb_file_s3_uri:{0}", tiledb_file_s3_uri);

### Delete the file array if exists

In [ ]:
var arrayinfo = TileDB.Cloud.RestUtil.GetArrayInfo(user_details.Username, tiledb_file_s3_uri);
if(arrayinfo != null) {
    System.Console.WriteLine("{0}", arrayinfo.ToJson());
    System.Console.WriteLine("Now start to delete {0}",tiledb_file_s3_uri );
    TileDB.Cloud.RestUtil.DeleteArray(user_details.Username, tiledb_file_s3_uri, "application/json");
}
else {
    System.Console.WriteLine("can not find uri: {0}", tiledb_file_s3_uri);
}
 

### Try to create a file array

In [ ]:
var file_details = TileDB.Cloud.RestUtil.CreateFile(user_details.Username, original_file, tiledb_file_s3_uri, tiledb_file_name);
if(file_details == null) 
{
    System.Console.WriteLine("failed to create file:{0}", tiledb_file_s3_uri);
} 
else {
    System.Console.WriteLine("{0}", file_details.ToJson());
}

In [ ]:
arrayinfo = TileDB.Cloud.RestUtil.GetArrayInfo(user_details.Username,tiledb_file_s3_uri);
if(arrayinfo==null)
{
    System.Console.WriteLine("failed to get array info for uri:{0}",tiledb_file_s3_uri);
}
else {
    System.Console.WriteLine("{0}",arrayinfo.ToJson());
}

### Add metadata

In [ ]:
using TileDB; //namespace using, need it to use extension methods
var ctx = new TileDB.Context();
//if you donot use using statement block, please call array_write.close() at the end to materialize it on disk
using(var array_write = new TileDB.Array(ctx, tiledb_file_s3_uri, TileDB.QueryType.TILEDB_WRITE))
{
    // add int metadata
    List<int> int_metadata = new List<int>() {1,100};
    array_write.AddArrayMetadataByList<int>("int_metadata",int_metadata);
    // add double metadata
    var double_metadata = new List<double>() {1.0,100.0,1000.0};
    array_write.AddArrayMetadataByList<double>("double_metadata", double_metadata);
    // add string key value pair 
    array_write.AddArrayMetadataByStringKeyValue("key1","value1");
    // add multiple string key value pairs 
    var key_value_map = new System.Collections.Generic.Dictionary<string,string>();
    key_value_map["key2"] = "value2";
    key_value_map["key3"] = "value3";
    array_write.AddArrayMetadataByStringMap(key_value_map);    

}


### Get metadata

#### open the array to read

In [ ]:
var array_read = new TileDB.Array(ctx,tiledb_file_s3_uri, TileDB.QueryType.TILEDB_READ);

#### get metadata from index

In [ ]:
var metadata_json = array_read.GetArrayMetadataJsonFromIndex(0);
System.Console.WriteLine("{0}", metadata_json.ToString());

#### get metadata from key

In [ ]:
var double_metadata_json = array_read.GetArrayMetadataJsonForKey("double_metadata");
System.Console.WriteLine("{0}", double_metadata_json.ToString());
var double_list = Newtonsoft.Json.JsonConvert.DeserializeObject<List<double>>(double_metadata_json["value"].ToString());
System.Console.WriteLine("type of double_list:{0}, value of double_list:{1}",double_list.GetType(),string.Join(",",double_list));

In [ ]:
// get string metadata
var string_metadata_json = array_read.GetArrayMetadataJsonForKey("key2");
System.Console.WriteLine("{0}", string_metadata_json.ToString());
// access key and value
System.Console.WriteLine("{0}:{1}",string_metadata_json["key"],string_metadata_json["value"]);

#### get all metadata

In [ ]:

var all_metadata_json = array_read.GetArrayMetadataJson();
System.Console.WriteLine("{0}", all_metadata_json.ToString());

### Catalog searching
        // Parameters:
        //   username: string
        //     namespace (optional)
        //
        //   permissions: string
        //     permissions valid values include read, read_write, write, admin (optional)
        //
        //   tags: List<string>
        //     tag to search for, more than one can be included (optional)
        //
        //   exclude_tags: List<string>
        //     tags to exclude matching array in results, more than one can be included (optional)
        //
        //   search: string
        //     search string that will look at name, namespace or description fields (optional)
        //
        //   file_types: List<string>
        //     file_type to search for, more than one can be included (optional)
        //
        //   exclude_file_types: List<string>
        //     file_type to exclude matching array in results, more than one can be included
        //     (optional)
        //
        //   page: int
        //     pagination offset (optional)
        //
        //   perPage: int
        //     pagination limit (optional)
        //
        // Returns:
        //     ArrayBrowserData

In [ ]:

List<string> file_types = new List<string>();
file_types.Add(TileDB.Cloud.Rest.Model.FileType.File.ToString());

//List owned arrays
var arrayList = TileDB.Cloud.RestUtil.ListArrays(username:user_details.Username,file_types:file_types);
Console.WriteLine("{0}", arrayList.ToJson());

//List public arrays
var listPublicArrays = TileDB.Cloud.RestUtil.ListPublicArrays(username:user_details.Username,file_types:file_types);
Console.WriteLine("{0}", listPublicArrays.ToJson());

//List shared arrays
var listSharedArrays = TileDB.Cloud.RestUtil.ListSharedArrays(username:user_details.Username, file_types:file_types);
Console.WriteLine("{0}", listSharedArrays.ToJson());

### Export from tiledb to s3

In [ ]:
string export_s3_uri = user_details.DefaultS3Path + "/" + "exported.pdf";

var file_exported = TileDB.Cloud.RestUtil.ExportFile(user_details.Username, tiledb_file_s3_uri, export_s3_uri);
if (file_exported == null)
{
    System.Console.WriteLine("failed export from {0} to {1}", tiledb_file_s3_uri, export_s3_uri);
}
else
{
    System.Console.WriteLine("{0}", file_exported.ToJson());
}


### Export between local file and cloud array

In [ ]:
//default vfs.s3.region is us-east-1
//This is an example for using custom config
TileDB.Config cfg = new TileDB.Config();
cfg.set("vfs.s3.region", "us-east-2");
TileDB.Context ctx2 = new TileDB.Context(cfg);

//export file array to local file, The function in RestUtil will be deprecated
//TileDB.Cloud.RestUtil.ExportFileArrayToLocalFile(ctx2, "s3://stefan-region-test/image_tdb", "image_exported.tiff");
TileDB.CoreUtil.ExportArrayToFile(ctx2, "s3://stefan-region-test/image_tdb", "image_exported.tiff" );

//export local file image_exported.tiff
//TileDB.Cloud.RestUtil.SaveLocalFileToFileArray(ctx2, "s3://tiledb-bin/image_tdb", "image_exported.tiff","","");
TileDB.CoreUtil.SaveFileToArray(ctx2,"s3://tiledb-bin/image_tdb", "image_exported.tiff","","");